In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
flight_df=spark.read.format("csv")\
            .option("header","false")\
            .option("inferschema","false")\
            .option("mode","FAILFAST")\
            .load("/FileStore/tables/2010_summary.csv")

flight_df.show(5)

+-----------------+-------------------+-----+
|              _c0|                _c1|  _c2|
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df_header=spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","false")\
            .option("mode","FAILFAST")\
            .load("/FileStore/tables/2010_summary.csv")

flight_df_header.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df_header.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: string (nullable = true)



In [0]:
flight_df_header_schema=spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .option("mode","FAILFAST")\
            .load("/FileStore/tables/2010_summary.csv")

flight_df_header_schema.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df_header_schema.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



repartition and coalesce

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%fs
ls FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121,1728819970000
dbfs:/FileStore/tables/Json_data_with_extra_key.json,Json_data_with_extra_key.json,232,1728893223000
dbfs:/FileStore/tables/bucket_by_id/,bucket_by_id/,0,0
dbfs:/FileStore/tables/csv_write/,csv_write/,0,0
dbfs:/FileStore/tables/csv_write_repartition/,csv_write_repartition/,0,0
dbfs:/FileStore/tables/employee_data_set.csv,employee_data_set.csv,444,1728976179000
dbfs:/FileStore/tables/employee_datas_set.csv,employee_datas_set.csv,444,1728976071000
dbfs:/FileStore/tables/employee_dataset-1.csv,employee_dataset-1.csv,741,1728970535000
dbfs:/FileStore/tables/employee_dataset.csv,employee_dataset.csv,741,1728970427000
dbfs:/FileStore/tables/employee_file.csv,employee_file.csv,230,1729014160000


In [0]:
flight_df=spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .load("dbfs:/FileStore/tables/2010_summary.csv")

In [0]:
flight_df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [0]:
flight_df.count()

Out[5]: 255

In [0]:
flight_df.rdd.getNumPartitions()

Out[6]: 1

In [0]:
partitioned_flight_df=flight_df.repartition(4)

In [0]:
partitioned_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   63|
|          1|   64|
|          2|   64|
|          3|   64|
+-----------+-----+



repartiton based on column or specific column

In [0]:
partitioned_on_column=flight_df.repartition(300,"ORIGIN_COUNTRY_NAME")

make 300 partiton based on origin_country_name
we have 255 records and we try to make more partition like 300 but we have 255 then what it will do it will keep null value in some record or partition

In [0]:
partitioned_on_column.rdd.getNumPartitions()

Out[10]: 300

In [0]:
partitioned_on_column.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show(300)

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|    1|
|          2|    1|
|          7|    1|
|         10|    1|
|         13|    1|
|         15|    2|
|         16|    2|
|         19|    1|
|         21|    1|
|         22|    1|
|         28|    1|
|         31|    1|
|         39|    1|
|         42|    1|
|         43|    1|
|         44|    1|
|         45|    2|
|         48|    1|
|         53|    1|
|         54|    1|
|         55|    1|
|         65|    1|
|         70|    1|
|         73|    1|
|         75|    1|
|         76|    2|
|         81|    1|
|         84|    2|
|         86|    1|
|         87|    1|
|         90|    1|
|         91|    1|
|         97|    2|
|        100|    1|
|        103|    2|
|        104|    1|
|        108|    1|
|        112|    2|
|        115|    1|
|        117|    2|
|        126|    1|
|        127|    2|
|        129|    1|
|        130|    2|
|        132|    1|
|        133|    1|
|        138|    1|


coalesce

In [0]:
coalesce_flight_df=flight_df.repartition(8)

In [0]:
coalesce_flight_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   32|
|          1|   31|
|          2|   32|
|          3|   32|
|          4|   32|
|          5|   32|
|          6|   32|
|          7|   32|
+-----------+-----+



In [0]:
three_coalesce_df=coalesce_flight_df.coalesce(3)

In [0]:
three_coalesce_df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|   63|
|          1|   96|
|          2|   96|
+-----------+-----+

